# Customization

In [1]:
from datetime import datetime, timezone
from pydantic import BaseModel, field_validator

In [8]:
class Meeting(BaseModel):
    when: datetime

    @field_validator('when', mode='wrap')
    def when_now(cls, input_value, handler):
        # If inputs is 'now' return the current time
        if input_value == 'now':
            return datetime.now()
        # else pass the value to the handler
        when = handler(input_value)
        return when


In [15]:
print(Meeting(when='2020-01-01T12:00+02:00'))

when=datetime.datetime(2020, 1, 1, 12, 0, tzinfo=TzInfo(+02:00))


In [16]:
print(Meeting(when='now'))

when=datetime.datetime(2024, 4, 1, 20, 41, 35, 326871)


## Validators with Annotated

In [79]:
import json
from typing import Any, List, Annotated
from pydantic import ValidatorFunctionWrapHandler, ValidationInfo, ValidationError
from pydantic.functional_validators import AfterValidator, BeforeValidator, WrapValidator

In [70]:
def check_squares(v: int) -> int:
    assert v**0.5 % 1 == 0, f'{v} is not a square number'
    return v


def double(v: Any) -> Any:
    return v * 2

In [71]:
# First we calculate the double and then the result of double goes in the check_squares
MyNumber = Annotated[int, AfterValidator(double), AfterValidator(check_squares)]

In [72]:
class DemoModel(BaseModel):
    number: List[MyNumber]

In [73]:
print(DemoModel(number=[2, 8]))

number=[4, 16]


Wrap Validator

In [75]:
def maybe_strip_whitespace(
    v: Any, handler: ValidatorFunctionWrapHandler, info: ValidationInfo
) -> int:
    if info.mode == 'json':
        assert isinstance(v, str), 'In JSON mode the input must be a string!'
        # you can call the handler multiple times
        try:
            return handler(v)
        except ValidationError:
            return handler(v.strip())
    assert info.mode == 'python'
    assert isinstance(v, int), 'In Python mode the input must be an int!'
    # do no further validation
    return v

In [76]:
MyNumber = Annotated[int, WrapValidator(maybe_strip_whitespace)]

In [77]:
class DemoModel(BaseModel):
    number: List[MyNumber]

In [78]:
print(DemoModel(number=[2, 8]))

number=[2, 8]


In [80]:
print(DemoModel.model_validate_json(json.dumps({'number': [' 2 ', '8']})))

number=[2, 8]


Field Validator & Model Validator

In [135]:
from pydantic import field_validator, model_validator
from typing import Union

In [137]:
class UserModel(BaseModel):
    name: str
    id: int
    password1: str
    password2: str

    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    # you can select multiple fields, or use '*' to select all fields
    @field_validator('id')
    @classmethod
    def check_alphanumeric(cls, v: int) -> int:
        if v > 10:
            if v % 2 == 0:
                raise ValueError('must be odd')
        return v
    
    @model_validator(mode='before')
    @classmethod
    def check_passwords_match(cls, data):
        if data['password1'] != data['password2']:
            raise ValueError('passwords do not match')
        return data


In [139]:
print(UserModel(name='Vasilis Mpletsos', id=10, password1='1234', password2='1234'))

name='Vasilis Mpletsos' id=10 password1='1234' password2='1234'
